In [24]:
PASTA = 'dados'

In [25]:
import pandas as pd

df = pd.read_parquet(f'{PASTA}/1.limpo.parquet').astype({'data': 'datetime64[ns]'})
len(df)

10298930

In [26]:
def remover_colunas(df):
  df.drop(columns=["Recuperadosnovos", "emAcompanhamentoNovos"], inplace=True)
  return df

In [27]:
def obter_estacao(data):
    ano = data.year
    datas_estacoes = {
        "verao": pd.Timestamp(f"{ano}-12-21"),
        "outono": pd.Timestamp(f"{ano}-3-21"),
        "inverno": pd.Timestamp(f"{ano}-6-21"),
        "primavera": pd.Timestamp(f"{ano}-9-21"),
    }

    if data >= datas_estacoes["verao"] or data < datas_estacoes["outono"]:
        return "Verão"
    elif data >= datas_estacoes["outono"] and data < datas_estacoes["inverno"]:
        return "Outono"
    elif data >= datas_estacoes["inverno"] and data < datas_estacoes["primavera"]:
        return "Inverno"
    else:
        return "Primavera"

def traduzir_dias_da_semana(ingles):
  portugues = {
     "Sunday": "Domingo",
     "Monday": "Segunda-feira",
     "Tuesday": "Terça-feira",
     "Wednesday": "Quarta-feira",
     "Thursday": "Quinta-feira",
     "Friday": "Sexta-feira",
     "Saturday": "Sábado"
  }

  return portugues[ingles]

def traduzir_mes(ingles):
  portuguese = {
      "January": "Janeiro",
      "February": "Fevereiro",
      "March": "Março",
      "April": "Abril",
      "May": "Maio",
      "June": "Junho",
      "July": "Julho",
      "August": "Agosto",
      "September": "Setembro",
      "October": "Outubro",
      "November": "Novembro",
      "December": "Dezembro"
  }

  return portuguese[ingles]

def adicionar_feature_datas(df):
  df["mes"] = df.data.dt.month_name()
  df["mes_traduzido"] = df["mes"].apply(traduzir_mes)
  df["dia_semana"] = df.data.dt.day_name()
  df["dia_semana_traduzido"] = df["dia_semana"].apply(traduzir_dias_da_semana)
  df["estacao"] = df["data"].apply(obter_estacao)

  return df

In [28]:
colunas = {
    'regiao': 'regiao',
    'estado': 'estado',
    'municipio': 'municipio',
    'coduf': 'coduf',
    'codmun': 'codmun',
    'codRegiaoSaude': 'cod_regiao_saude',
    'nomeRegiaoSaude': 'nome_regiao_saude',
    'data': 'data',
    'semanaEpi': 'semana_epi',
    'populacaoTCU2019': 'populacao_tcu_2019',
    'casosAcumulado': 'casos_acumulados',
    'casosNovos': 'casos_novos',
    'novos_casos_novos': 'novos_casos_novos',
    'obitosAcumulado': 'obitos_acumulados',
    'obitosNovos': 'obitos_novos',
    'Recuperadosnovos': 'recuperados_novos',
    'emAcompanhamentoNovos': 'em_acompanhamento_novos',
    'interior/metropolitana': 'interior_metropolitana',
    'novos_casos_acumulados': 'novos_casos_acumulados'
}

def limpar_nomes_de_colunas(df):
  return df.rename(columns=colunas)

In [29]:
def rodar_engenharia_de_atributos(df):
  df = remover_colunas(df)
  df = adicionar_feature_datas(df)
  df = limpar_nomes_de_colunas(df)

  return df


df = rodar_engenharia_de_atributos(df)
len(df)

10298930

In [30]:
df.columns

Index(['regiao', 'estado', 'municipio', 'coduf', 'codmun', 'cod_regiao_saude',
       'nome_regiao_saude', 'data', 'semana_epi', 'populacao_tcu_2019',
       'casos_acumulados', 'casos_novos', 'obitos_acumulados', 'obitos_novos',
       'interior_metropolitana', 'novos_casos_novos', 'novos_casos_acumulados',
       'mes', 'mes_traduzido', 'dia_semana', 'dia_semana_traduzido',
       'estacao'],
      dtype='object')

In [31]:
df.to_parquet(f'{PASTA}/2.atributos.parquet', index=False)